In [11]:
import os
import shutil
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import re
from collections import defaultdict

from pymatgen.core.structure import Structure
from pymatgen.io.cif import CifWriter

from get_weirdos_ver3_new_cleaned import apply_pbc, mic_eucledian_distance

# TO DO: 

fullness needs repairment for the dictionary of weirdos_appended!!!!

1) create folder manually with file of geometry 0 and 1 
2) copy manually the necessary file there

In [12]:
path_geo_0 = [0, 1, 2, 3, 4, 5]
path_geo_1 = [0, 1, 2, 3]
path_geo_2 = [0, 1, 2, 3, 4]
path_geo_3 = [0, 1, 2, 3, 4, 5, 6, 7]
path_geo_4 = [0, 1, 2, 3, 4, 5]
path_geo_5 = [0, 1, 2, 3, 4, 5, 6, 7]
path_geo_6 = [0, 1, 2, 3, 4, 5, 6, 7]
path_geo_7 = [0, 1, 2, 3, 4, 5]
path_geo_8 = [0, 1, 2, 3]

In [13]:
# def get_idxs_Li(file_path):
#     # Read the content of the file
#     with open(file_path, 'r') as f:
#         content = f.read()

#     # Use regular expressions to find the indices of "Li" entries
#     idxs_Li = [int(match.group(1)) for match in re.finditer(r'Li\s+Li(\d+)', content)]

#     return idxs_Li


def get_idx_coor_Li_dict(file_path):
    with open(file_path, 'r') as f:
        content = f.read()

    # Initialize a dictionary to store the data
    Li_idx_coor_dict = {}

    # Use regular expressions to extract Li indices and coordinates
    li_pattern = re.compile(r'Li\s+Li(\d+)\s+1\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)')
    matches = li_pattern.findall(content)

    # Iterate through the matches and populate the dictionary
    for match in matches:
        index = int(match[0])
        x = float(match[1])
        y = float(match[2])
        z = float(match[3])
        Li_idx_coor_dict[index] = (x, y, z)

    return Li_idx_coor_dict


def get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict):
    idx_coor_Li_idx_centroid_triad = defaultdict(list)
    for key, coor in idx_coor_Li_dict.items():
        idx_coor_Li_idx_centroid_triad_dict = {}    
        for key_triad, coor_triad in idx_coor_Li_dict_ref_triad.items():
            for coor_triad_component in coor_triad:
                distance = mic_eucledian_distance(coor_triad_component, coor)
                if distance == 0:
                    idx_coor_Li_idx_centroid_triad_dict['coor'] = coor
                    idx_coor_Li_idx_centroid_triad_dict['idx_triad'] = key_triad
                    idx_coor_Li_idx_centroid_triad_dict['centroid_triad'] = coor_triad[0]
                    # check if it's at 24g or 48h
                    if coor == coor_triad[0]:
                        idx_coor_Li_idx_centroid_triad_dict['structure'] = 24
                    else:
                        idx_coor_Li_idx_centroid_triad_dict['structure'] = 48                    
                    if key in idx_coor_Li_idx_centroid_triad:
                        idx_coor_Li_idx_centroid_triad[key].append(idx_coor_Li_idx_centroid_triad_dict)
                    else:
                        idx_coor_Li_idx_centroid_triad[key] = idx_coor_Li_idx_centroid_triad_dict
    return idx_coor_Li_idx_centroid_triad


# def get_24g_or_48h(idx_coor_Li_idx_centroid_triad):
#     # idxs_Li_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad.keys()]
#     idx_coor_Li_idx_centroid_triad = defaultdict(list)
#     for key, val in enumerate(idx_coor_Li_idx_centroid_triad):
#         if val['coor'] == val['centroid_triad']:


#     # idx_coor_Li_idx_centroid_triad = defaultdict(list)
#     # for key, coor in idx_coor_Li_dict.items():
#     #     idx_coor_Li_idx_centroid_triad_dict = {}    
#     #     for key_triad, coor_triad in idx_coor_Li_dict_ref_triad.items():
#     #         for coor_triad_component in coor_triad:
#     #             distance = mic_eucledian_distance(coor_triad_component, coor)
#     #             if distance == 0:
#     #                 idx_coor_Li_idx_centroid_triad_dict['coor'] = coor
#     #                 idx_coor_Li_idx_centroid_triad_dict['idx_triad'] = key_triad
#     #                 idx_coor_Li_idx_centroid_triad_dict['centroid_triad'] = coor_triad[0]                
#     #                 if key in idx_coor_Li_idx_centroid_triad:
#     #                     idx_coor_Li_idx_centroid_triad[key].append(idx_coor_Li_idx_centroid_triad_dict)
#     #                 else:
#     #                     idx_coor_Li_idx_centroid_triad[key] = idx_coor_Li_idx_centroid_triad_dict
#     return idx_coor_Li_idx_centroid_triad


def get_dist_ascending(idx_coor_Li_idx_centroid_triad):
    coors_Li_dist_structures = defaultdict(list)

    for Li_idx_temp1, val_temp1 in idx_coor_Li_idx_centroid_triad.items():
        coors_Li_dist_structures_dict = {}
        for Li_idx_temp2, val_temp2 in idx_coor_Li_idx_centroid_triad.items():
            distance = mic_eucledian_distance(val_temp1['coor'], val_temp2['coor'])
            coors_Li_dist_structures_dict['coors'] = (val_temp1['coor'], val_temp2['coor'])
            coors_Li_dist_structures_dict['dist'] = distance
            coors_Li_dist_structures_dict['structures'] = (val_temp1['structure'], val_temp2['structure'])

            key = (Li_idx_temp1, Li_idx_temp2)
            if key in coors_Li_dist_structures:
                coors_Li_dist_structures[key].append(coors_Li_dist_structures_dict)
            else:
                coors_Li_dist_structures[key] = coors_Li_dist_structures_dict

    sorted_coors_Li_dist_structures = dict(sorted(coors_Li_dist_structures.items(), key=lambda item: item[1]['dist']))
    dist_ascending = list({val['dist'] for idx, val in sorted_coors_Li_dist_structures.items()})

    return dist_ascending, sorted_coors_Li_dist_structures


def get_idx_coor_Li_idx_centroid_triad_w_closest_dist(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict):
    idx_coor_Li_idx_centroid_triad = defaultdict(list)
    distance_array = []
    for key, coor in idx_coor_Li_dict.items():
        idx_coor_Li_idx_centroid_triad_dict = {}    
        for key_triad, coor_triad in idx_coor_Li_dict_ref_triad.items():
            for coor_triad_component in coor_triad:
                distance = mic_eucledian_distance(coor_triad_component, coor)
                distance_array.append(distance)
                if distance == 0:
                    idx_coor_Li_idx_centroid_triad_dict['coor'] = coor
                    idx_coor_Li_idx_centroid_triad_dict['idx_triad'] = key_triad
                    idx_coor_Li_idx_centroid_triad_dict['centroid_triad'] = coor_triad[0]
                    # check if it's at 24g or 48h
                    if coor == coor_triad[0]:
                        idx_coor_Li_idx_centroid_triad_dict['structure'] = 24
                    else:
                        idx_coor_Li_idx_centroid_triad_dict['structure'] = 48                    
        distance_array_sorted = sorted(distance_array)
        distance_array_sorted_top3 = distance_array_sorted[1:4]
        idx_coor_Li_idx_centroid_triad_dict['dist_top3'] = distance_array_sorted_top3
        # idx_coor_Li_idx_centroid_triad_dict['dist'] = distance_array_sorted
        if key in idx_coor_Li_idx_centroid_triad:
            idx_coor_Li_idx_centroid_triad[key].append(idx_coor_Li_idx_centroid_triad_dict)
        else:
            idx_coor_Li_idx_centroid_triad[key] = idx_coor_Li_idx_centroid_triad_dict
    return idx_coor_Li_idx_centroid_triad


# def get_idx_coor_Li_idx_centroid_triad_w_closest_dist_weirdos_appended(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended, idxs_Li_not):
#     idx_coor_Li_idx_centroid_triad = defaultdict(list)
#     distance_array = []
#     for key, coor in idx_coor_Li_dict_weirdos_appended.items():
#         if key in idxs_Li_not:
#             idx_coor_Li_idx_centroid_triad_dict = {}    
#             for key_triad, coor_triad in idx_coor_Li_dict_ref_triad.items():
#                 for coor_triad_component in coor_triad:
#                     distance = mic_eucledian_distance(coor_triad_component, coor)
#                     distance_array.append(distance)  
#             distance_array_sorted = sorted(set(distance_array))
#             distance_array_sorted_top3 = distance_array_sorted[0:4]
#             idx_coor_Li_idx_centroid_triad_dict['dist_top3'] = distance_array_sorted_top3           
#         else:
#             idx_coor_Li_idx_centroid_triad_dict = {}    
#             for key_triad, coor_triad in idx_coor_Li_dict_ref_triad.items():
#                 for coor_triad_component in coor_triad:
#                     distance = mic_eucledian_distance(coor_triad_component, coor)
#                     distance_array.append(distance)
#                     if distance == 0:
#                         idx_coor_Li_idx_centroid_triad_dict['coor'] = coor
#                         idx_coor_Li_idx_centroid_triad_dict['idx_triad'] = key_triad
#                         idx_coor_Li_idx_centroid_triad_dict['centroid_triad'] = coor_triad[0]
#                         # check if it's at 24g or 48h
#                         if coor == coor_triad[0]:
#                             idx_coor_Li_idx_centroid_triad_dict['structure'] = 24
#                         else:
#                             idx_coor_Li_idx_centroid_triad_dict['structure'] = 48           
#         distance_array_sorted = sorted(set(distance_array))
#         distance_array_sorted_top3 = distance_array_sorted[0:4]
#         idx_coor_Li_idx_centroid_triad_dict['dist_top3'] = distance_array_sorted_top3
#         if key in idx_coor_Li_idx_centroid_triad:
#             idx_coor_Li_idx_centroid_triad[key].append(idx_coor_Li_idx_centroid_triad_dict)
#         else:
#             idx_coor_Li_idx_centroid_triad[key] = idx_coor_Li_idx_centroid_triad_dict
#     return idx_coor_Li_idx_centroid_triad


def get_idx_coor_Li_idx_centroid_triad_w_closest_dist_weirdos_appended(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended, idxs_Li_not):
    idx_coor_Li_idx_centroid_triad_weirdos_appended = defaultdict(list)
    distance_array = []
    for key, coor in idx_coor_Li_dict_weirdos_appended.items():
        # if key in idxs_Li_not:
        #     idx_coor_Li_idx_centroid_triad_dict = {}    
        #     for key_triad, coor_triad in idx_coor_Li_dict_ref_triad.items():
        #         for coor_triad_component in coor_triad:
        #             distance = mic_eucledian_distance(coor_triad_component, coor)
        #             distance_array.append(distance)  
        #     distance_array_sorted = sorted(set(distance_array))
        #     distance_array_sorted_top3 = distance_array_sorted[0:4]
        #     idx_coor_Li_idx_centroid_triad_dict['dist_top3'] = distance_array_sorted_top3  
        #     if key in idx_coor_Li_idx_centroid_triad_weirdos_appended:
        #         idx_coor_Li_idx_centroid_triad_weirdos_appended[key].append(idx_coor_Li_idx_centroid_triad_dict)
        #     else:
        #         idx_coor_Li_idx_centroid_triad_weirdos_appended[key] = idx_coor_Li_idx_centroid_triad_dict         
        # else:
        idx_coor_Li_idx_centroid_triad_dict = {}    
        for key_triad, coor_triad in idx_coor_Li_dict_ref_triad.items():
            for coor_triad_component in coor_triad:
                distance = mic_eucledian_distance(coor_triad_component, coor)
                distance_array.append(distance)
                if distance == 0:
                    idx_coor_Li_idx_centroid_triad_dict['coor'] = coor
                    idx_coor_Li_idx_centroid_triad_dict['idx_triad'] = key_triad
                    idx_coor_Li_idx_centroid_triad_dict['centroid_triad'] = coor_triad[0]
                    # check if it's at 24g or 48h
                    if coor == coor_triad[0]:
                        idx_coor_Li_idx_centroid_triad_dict['structure'] = 24
                    else:
                        idx_coor_Li_idx_centroid_triad_dict['structure'] = 48           
        distance_array_sorted = sorted(set(distance_array))
        distance_array_sorted_top3 = distance_array_sorted[0:4]
        # idx_coor_Li_idx_centroid_triad_dict['dist_top3'] = distance_array_sorted_top3
        if key in idx_coor_Li_idx_centroid_triad_weirdos_appended:
            idx_coor_Li_idx_centroid_triad_weirdos_appended[key].append(idx_coor_Li_idx_centroid_triad_dict)
        else:
            idx_coor_Li_idx_centroid_triad_weirdos_appended[key] = idx_coor_Li_idx_centroid_triad_dict
    return idx_coor_Li_idx_centroid_triad_weirdos_appended

# find the distance of Li between CONCTAR_pymatgen of each path wrt path 0

In [14]:
def get_distance_wrtpath0_and_litoli(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename):
    # df_coor = pd.DataFrame()
    df_distance = pd.DataFrame()
    coor_Li_ref = []
    df_dist_litoli = pd.DataFrame()

    df_dist_litoli["dist"] = None

    # col_xyz_coor = "xyz_coor"

    # df_coor[col_xyz_coor] = None

    if geo == 0:
        path_geo = path_geo_0
    elif geo == 1:
        path_geo = path_geo_1
    elif geo == 2:
        path_geo = path_geo_2
    elif geo == 3:
        path_geo = path_geo_3
    elif geo == 4:
        path_geo = path_geo_4
    elif geo == 5:
        path_geo = path_geo_5
    elif geo == 6:
        path_geo = path_geo_6
    elif geo == 7:
        path_geo = path_geo_7
    elif geo == 8:
        path_geo = path_geo_8

    file_ref = f"{geo}_0_{var_filename_ref}.cif"
    file_path_ref = os.path.join(destination_directory, file_ref)

    structure_ref = Structure.from_file(file_path_ref)

    for idx, coor in enumerate(structure_ref):
        if coor.species_string == "Li":
            coor_Li_ref.append(coor.frac_coords)

    for i in path_geo:
        coor_Li = []
        file = f"{geo}_{i}_{var_filename}.cif"
        file_path = os.path.join(destination_directory, file)
    
        structure = Structure.from_file(file_path)
        # frac_coor = structure.frac_coords

        for idx, coor in enumerate(structure):
            if coor.species_string == "Li":
                coor_Li.append(coor.frac_coords)        
            
        # df_coor.at[i, col_xyz_coor] = coor_Li#

        coors_Li_dist_structures = defaultdict(list)

        for j in range(len(coor_Li)):
            # df_distance.at[i, f"{j}"] = None  

            distance = mic_eucledian_distance(coor_Li_ref[j], coor_Li[j])

            df_distance.at[i, f"{j}"] = distance

            diameter_24g48h = max_mapping_radius * 2
            if distance < diameter_24g48h and i != 0:
                print(f"path: {i}, Li: {j}, distance: {distance}")

            coors_Li_dist_structures_dict = {}
            
            for k in range(len(coor_Li)):

                distance_litoli = mic_eucledian_distance(coor_Li[j], coor_Li[k])

                coors_Li_dist_structures_dict['coors'] = (coor_Li[j], coor_Li[k])
                coors_Li_dist_structures_dict['dist'] = distance_litoli
                # coors_Li_dist_structures_dict['structures'] = (val_temp1['structure'], val_temp2['structure'])

                key = (j, k)
                if key in coors_Li_dist_structures:
                    coors_Li_dist_structures[key].append(coors_Li_dist_structures_dict)
                else:
                    coors_Li_dist_structures[key] = coors_Li_dist_structures_dict               

        sorted_coors_Li_dist_structures = dict(sorted(coors_Li_dist_structures.items(), key=lambda item: item[1]['dist']))
        dist_ascending = list({val['dist'] for idx, val in sorted_coors_Li_dist_structures.items()})

        df_dist_litoli.at[i, "dist"] = dist_ascending[1:6]

    return df_distance, df_dist_litoli

In [15]:
def get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_ref_24, idx_coor_Li_dict_ref_72):
    idxs_Li_ref_24 = list(idx_coor_Li_dict_ref_24.keys())
    idxs_Li_ref_72 = list(idx_coor_Li_dict_ref_72.keys())

    idx_coor_Li_dict_ref_triad = defaultdict(list)

    for key_72, coor_72 in idx_coor_Li_dict_ref_72.items():
        for key_24, coor_24 in idx_coor_Li_dict_ref_24.items():
            distance = mic_eucledian_distance(coor_72, coor_24)
            if distance == 0:
                idx_coor_Li_dict_ref_triad[key_24].append(coor_72)

    for key_72, coor_72 in idx_coor_Li_dict_ref_72.items():
        for key_24, coor_24 in idx_coor_Li_dict_ref_24.items():
            distance = mic_eucledian_distance(coor_72, coor_24)
            if distance <= 0.086399 and distance != 0:                      # to edit this number
                idx_coor_Li_dict_ref_triad[key_24].append(coor_72)
    return idx_coor_Li_dict_ref_triad

In [16]:
def get_distance_wrtpath0(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename):
    # df_coor = pd.DataFrame()
    df_distance = pd.DataFrame()
    coor_Li_ref = []

    # col_xyz_coor = "xyz_coor"

    # df_coor[col_xyz_coor] = None

    if geo == 0:
        path_geo = path_geo_0
    elif geo == 1:
        path_geo = path_geo_1
    elif geo == 2:
        path_geo = path_geo_2
    elif geo == 3:
        path_geo = path_geo_3
    elif geo == 4:
        path_geo = path_geo_4
    elif geo == 5:
        path_geo = path_geo_5
    elif geo == 6:
        path_geo = path_geo_6
    elif geo == 7:
        path_geo = path_geo_7
    elif geo == 8:
        path_geo = path_geo_8

    file_ref = f"{geo}_0_{var_filename_ref}.cif"
    file_path_ref = os.path.join(destination_directory, file_ref)

    structure_ref = Structure.from_file(file_path_ref)

    for idx, coor in enumerate(structure_ref):
        if coor.species_string == "Li":
            coor_Li_ref.append(coor.frac_coords)

    for i in path_geo:
        coor_Li = []
        file = f"{geo}_{i}_{var_filename}.cif"
        file_path = os.path.join(destination_directory, file)
    
        structure = Structure.from_file(file_path)
        # frac_coor = structure.frac_coords

        for idx, coor in enumerate(structure):
            if coor.species_string == "Li":
                coor_Li.append(coor.frac_coords)        
            
        # df_coor.at[i, col_xyz_coor] = coor_Li

        for j in range(len(coor_Li)):
            # df_distance.at[i, f"{j}"] = None  

            distance = mic_eucledian_distance(coor_Li_ref[j], coor_Li[j])

            df_distance.at[i, f"{j}"] = distance

            diameter_24g48h = max_mapping_radius * 2
            if distance < diameter_24g48h and i != 0:
                print(f"path: {i}, Li: {j}, distance: {distance}")

    return df_distance


def plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs):

    diameter_24g48h = max_mapping_radius * 2

    x = range(len(df_distance))

    fig = plt.figure()
    ax = plt.subplot(111)

    lines = []

    for i in range(len(df_distance.columns)):
        if Li_idxs == "all" or i in Li_idxs:
            # # i = i
            line, = ax.plot(x, df_distance[f"{i}"], label=f"{i}")
            lines.append(line)
            # label = f"{i}" if Li_idxs == "all" else None
            # line, = ax.plot(x, df_distance[f"{i}"], label=label)
            lines.append(line)

        # if type(Li_idxs) == list:
        #     for j in Li_idxs:
        #         if i == j:
        #             line, = ax.plot(x, df_distance[f"{i}"], label=f"{i}")
        #             lines.append(line)

    ax.axhline(y=diameter_24g48h, color='b', linestyle=':', label=f'd_24g48h = {diameter_24g48h:.3f}')

    plt.title(f"Geometry {geo} with d={diameter_24g48h}")

    # Shrink current axis's height by 10% on the bottom
        # source: https://stackoverflow.com/questions/4700614/how-to-put-the-legend-outside-the-plot
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                    box.width, box.height * 0.9])

    handles, labels = ax.get_legend_handles_labels()

    ax.legend(handles=handles, labels=labels, loc='upper center', bbox_to_anchor=(0.5, -0.05),
            fancybox=True, shadow=True, ncol=5)

    plt.show()

In [17]:
def get_distance_triads_wrtpath0(destination_directory, geo, var_filename, filename_ref_72):
    # df_coor = pd.DataFrame()
    df_distance = pd.DataFrame()
    df_ratio = pd.DataFrame()
    # coor_Li_ref = []

    # col_xyz_coor = "xyz_coor"

    # df_coor[col_xyz_coor] = None

    if geo == 0:
        path_geo = path_geo_0
    elif geo == 1:
        path_geo = path_geo_1
    elif geo == 2:
        path_geo = path_geo_2
    elif geo == 3:
        path_geo = path_geo_3
    elif geo == 4:
        path_geo = path_geo_4
    elif geo == 5:
        path_geo = path_geo_5
    elif geo == 6:
        path_geo = path_geo_6
    elif geo == 7:
        path_geo = path_geo_7
    elif geo == 8:
        path_geo = path_geo_8

    file_ref_24 = f"{geo}_0_{var_filename}.cif"
    file_path_ref_24 = os.path.join(destination_directory, file_ref_24)

    file_ref_72 = f"{filename_ref_72}.cif"
    file_path_ref_72 = os.path.join(destination_directory, file_ref_72)


    idx_coor_Li_dict_ref_24 = get_idx_coor_Li_dict(file_path_ref_24)    # key is the pointer to 24
    idx_coor_Li_dict_ref_72 = get_idx_coor_Li_dict(file_path_ref_72)    # key is the pointer to 24

    idx_coor_Li_dict_ref_triad = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_ref_24, idx_coor_Li_dict_ref_72)
    # idxs_Li_ref_24 = list(idx_coor_Li_dict_ref_24.keys())
    # idxs_Li_ref_72 = list(idx_coor_Li_dict_ref_72.keys())

    for i in path_geo:
        # coor_Li = []
        file = f"{geo}_{i}_{var_filename}.cif"
        file_path = os.path.join(destination_directory, file)

        idx_coor_Li_dict = get_idx_coor_Li_dict(file_path)
        # idxs_Li = list(idx_coor_Li_dict.keys())

        # # idx_coor_Li_triad_belonging_initial = defaultdict(list)
        # # idx_coor_Li_triad_belonging_initial_centroid = defaultdict(list)

        ### does the numeration of Li is important?
        ### 1) check which triad it does belong to initially

        idx_coor_Li_idx_centroid_triad_ref = get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_ref_24)
        idx_coor_Li_idx_centroid_triad = get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict)
        idxs_Li_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad.keys()]

        ## get ratio of 24:48
        counter_48 = 0
        for Li_idx, val in idx_coor_Li_idx_centroid_triad.items():
            # print(mic_eucledian_distance(val['coor'], val['centroid_triad']))
            if val['structure'] == 48:
                counter_48 = counter_48 + 1
        # print(f"path {i} has ratio of 48 of: {counter_48/len(idx_coor_Li_idx_centroid_triad)}")
        df_ratio.at[i, "ratio of 48"] = counter_48/len(idx_coor_Li_idx_centroid_triad)

        for j in idxs_Li_dict:
            # df_distance.at[i, f"{j}"] = None  

            distance = mic_eucledian_distance(idx_coor_Li_idx_centroid_triad_ref[j]['centroid_triad'], idx_coor_Li_idx_centroid_triad[j]["centroid_triad"])

            df_distance.at[i, f"{j}"] = distance

            if distance == 0 and i != 0:
                print(f"path: {i}, Li: {j}, distance: {distance}")

    return df_distance, df_ratio

In [18]:
def get_triads_movement(destination_directory, geo, var_filename, filename_ref_72):
    # df_coor = pd.DataFrame()
    df_triad = pd.DataFrame()
    df_ratio = pd.DataFrame()
    df_dist = pd.DataFrame()

    df_dist["dist"] = None
    # coor_Li_ref = []

    # col_xyz_coor = "xyz_coor"

    # df_coor[col_xyz_coor] = None

    if geo == 0:
        path_geo = path_geo_0
    elif geo == 1:
        path_geo = path_geo_1
    elif geo == 2:
        path_geo = path_geo_2
    elif geo == 3:
        path_geo = path_geo_3
    elif geo == 4:
        path_geo = path_geo_4
    elif geo == 5:
        path_geo = path_geo_5
    elif geo == 6:
        path_geo = path_geo_6
    elif geo == 7:
        path_geo = path_geo_7
    elif geo == 8:
        path_geo = path_geo_8

    file_ref_24 = f"{geo}_0_{var_filename}.cif"
    file_path_ref_24 = os.path.join(destination_directory, file_ref_24)

    file_ref_72 = f"{filename_ref_72}.cif"
    file_path_ref_72 = os.path.join(destination_directory, file_ref_72)


    idx_coor_Li_dict_ref_24 = get_idx_coor_Li_dict(file_path_ref_24)    # key is the pointer to 24
    idx_coor_Li_dict_ref_72 = get_idx_coor_Li_dict(file_path_ref_72)    # key is the pointer to 24

    idx_coor_Li_dict_ref_triad = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_ref_24, idx_coor_Li_dict_ref_72)
    # idxs_Li_ref_24 = list(idx_coor_Li_dict_ref_24.keys())
    # idxs_Li_ref_72 = list(idx_coor_Li_dict_ref_72.keys())

    for i in path_geo:
        # coor_Li = []
        file = f"{geo}_{i}_{var_filename}.cif"
        file_path = os.path.join(destination_directory, file)

        idx_coor_Li_dict = get_idx_coor_Li_dict(file_path)
        # idxs_Li = list(idx_coor_Li_dict.keys())

        # # idx_coor_Li_triad_belonging_initial = defaultdict(list)
        # # idx_coor_Li_triad_belonging_initial_centroid = defaultdict(list)

        ### does the numeration of Li is important?
        ### 1) check which triad it does belong to initially

        idx_coor_Li_idx_centroid_triad_ref = get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_ref_24)
        idx_coor_Li_idx_centroid_triad = get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict)
        idxs_Li_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad.keys()]

        ## get ratio of 24:48
        counter_48 = 0
        for Li_idx, val in idx_coor_Li_idx_centroid_triad.items():
            # print(mic_eucledian_distance(val['coor'], val['centroid_triad']))
            if val['structure'] == 48:
                counter_48 = counter_48 + 1
        # print(f"path {i} has ratio of 48 of: {counter_48/len(idx_coor_Li_idx_centroid_triad)}")
        df_ratio.at[i, "ratio of 48"] = counter_48/len(idx_coor_Li_idx_centroid_triad)

        ## get li-to-li-distance 
        dist_ascending, sorted_coors_Li_dist_structures = get_dist_ascending(idx_coor_Li_idx_centroid_triad)
        # print(dist_ascending)
        df_dist.at[i, "dist"] = dist_ascending[1:6]

        for j in idxs_Li_dict:
            # df_triad.at[i, f"{j}"] = None  

            triad = idx_coor_Li_idx_centroid_triad[j]["idx_triad"]

            df_triad.at[i, f"{j}"] = triad

            if triad == df_triad.at[0, f"{j}"] and i != 0:
                print(f"path: {i}, Li: {j}, triad: {triad}")

    return df_triad, df_ratio, df_dist, sorted_coors_Li_dist_structures

# find the distance of Li between CONCTAR_pymatgen of each path wrt path 0

# CONTCAR to CONTCAR comparison

## max_mapping_radius = 0.043

Path 1: idx0_weirdos_Li = [[9, 21]]

Path 3: idx0_weirdos_Li = [[3, 12]

In [23]:
max_mapping_radius = 0.09
var_filename_ref = "CONTCAR_pymatgen"
var_filename = "CONTCAR_pymatgen"
file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_type5_trial"
destination_directory = f"/restructure_{max_mapping_radius}/"

In [25]:
geo = 0
df_distance, df_dist_litoli = get_distance_wrtpath0_and_litoli(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

FileNotFoundError: [Errno 2] No such file or directory: '/restructure_0.09_Li6PS5Cl_type5_trial/0_0_CONTCAR_pymatgen.cif'

In [21]:
df_dist_litoli

NameError: name 'df_dist_litoli' is not defined

In [ ]:
df_dist_litoli.to_excel("df_dist_litoli_0.xlsx")  

In [ ]:
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = [9, 21, 3, 12])

In [ ]:
geo = 1
df_distance, df_dist_litoli = get_distance_wrtpath0_and_litoli(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
df_dist_litoli

In [ ]:
df_dist_litoli.to_excel("df_dist_litoli_1.xlsx")  

## r = 0.01

In [ ]:
max_mapping_radius = 0.09
var_filename_ref = "CONTCAR_pymatgen"
var_filename = "CONTCAR_pymatgen"
destination_directory = f"restructure_{max_mapping_radius}/"

# CONTCAR to mapLi_reindexed_weirdos_appended_reordered comparison

## r = 0.043

In [ ]:
# max_mapping_radius = 0.043
# var_filename_ref = "CONTCAR_pymatgen"
# var_filename = "mapLi_reindexed_weirdos_appended_reordered"
# destination_directory = f"restructure_{max_mapping_radius}/"

In [ ]:
# geo = 0
# df_distance = get_distance_wrtpath0(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename)
# plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
# geo = 1
# df_distance = get_distance_wrtpath0(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename)
# plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

# Moving distance between triads

## just trials changing var_filename

In [ ]:
geo = 0
max_mapping_radius = 0.09
destination_directory = f"restructure_{max_mapping_radius}/"
var_filename_ref = "mapLi_reindexed"
var_filename = "mapLi_reindexed"

In [ ]:
geo = 0
df_distance = get_distance_wrtpath0(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
geo = 1
df_distance = get_distance_wrtpath0(destination_directory, geo, max_mapping_radius, var_filename_ref, var_filename)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

## using the real algorithm

## r = 0.043

In [ ]:
max_mapping_radius = 0.09
var_filename = "mapLi_reindexed"
filename_ref_72 = "Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA_expanded"

In [ ]:
geo = 0 
df_triad, df_ratio = get_distance_triads_wrtpath0(destination_directory, geo, var_filename, filename_ref_72)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
df_ratio

In [ ]:
df_distance.iloc[:, 0:12]

In [ ]:
df_distance.iloc[:, 12:24]

In [ ]:
geo = 1
df_triad, df_ratio = get_distance_triads_wrtpath0(destination_directory, geo, var_filename, filename_ref_72)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
df_ratio

In [ ]:
df_distance.iloc[:, 0:12]

In [ ]:
df_distance.iloc[:, 12:24]

## idx_triad

In [ ]:
geo = 0 
df_triad_0, df_ratio, df_dist, sorted_coors_Li_dist_structures = get_triads_movement(destination_directory, geo, var_filename, filename_ref_72)
plot_distance_wrtpath0(df_triad_0, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
df_ratio

In [ ]:
df_dist

In [ ]:
sorted_coors_Li_dist_structures

In [ ]:
df_triad_0.iloc[:, 0:12]

In [ ]:
df_triad_0.iloc[:, 12:24]

In [ ]:
plot_distance_wrtpath0(df_triad_0, geo, max_mapping_radius, Li_idxs = [1])

In [ ]:
geo = 1 
df_triad_1, df_ratio, df_dist, sorted_coors_Li_dist_structures = get_triads_movement(destination_directory, geo, var_filename, filename_ref_72)
plot_distance_wrtpath0(df_triad_1, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
df_ratio

In [ ]:
df_dist

In [ ]:
sorted_coors_Li_dist_structures

In [ ]:
df_triad_1.iloc[:, 0:12]

In [ ]:
df_triad_1.iloc[:, 12:24]

## idx_triad_normalized from starting point

In [ ]:
geo = 0

df_triad_0_normalized = df_triad_0.copy()
# for i in range(len(df_triad_0)):
for i in range(len(df_triad_0.columns)):
    init_triad = df_triad_0.iloc[0, i]
    df_triad_0_normalized.iloc[:, i] = df_triad_0_normalized.iloc[:, i] - init_triad
    # for j in range(len(df_triad_0.columns)):
    #     df_triad_0_normalized.iloc[i, j] = df_triad_0_normalized.iloc[i, j] - init_triad

In [ ]:
df_triad_0_normalized.iloc[:, 0:12]

In [ ]:
df_triad_0_normalized.iloc[:, 12:24]

In [ ]:
plot_distance_wrtpath0(df_triad_0_normalized, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
geo = 1

df_triad_1_normalized = df_triad_1.copy()
# for i in range(len(df_triad_1)):
for i in range(len(df_triad_1.columns)):
    init_triad = df_triad_1.iloc[0, i]
    df_triad_1_normalized.iloc[:, i] = df_triad_1_normalized.iloc[:, i] - init_triad

In [ ]:
plot_distance_wrtpath0(df_triad_1_normalized, geo, max_mapping_radius, Li_idxs = [3, 7])

In [ ]:
df_triad_1_normalized.iloc[:, 0:12]

In [ ]:
df_triad_1_normalized.iloc[:, 12:24]

In [ ]:
plot_distance_wrtpath0(df_triad_1_normalized, geo, max_mapping_radius, Li_idxs = "all")

## r = 0.055

In [ ]:
max_mapping_radius = 0.055
destination_directory = f"restructure_{max_mapping_radius}/"
var_filename = "mapLi_reindexed"
filename_ref_72 = "Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA_expanded"

In [ ]:
geo = 0 
df_distance, df_ratio = get_distance_triads_wrtpath0(destination_directory, geo, var_filename, filename_ref_72)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
geo = 1
df_distance, df_ratio = get_distance_triads_wrtpath0(destination_directory, geo, var_filename, filename_ref_72)
plot_distance_wrtpath0(df_distance, geo, max_mapping_radius, Li_idxs = "all")

## idx_triad_normalized from starting point

In [ ]:
geo = 0

df_triad_0_normalized = df_triad_0.copy()
# for i in range(len(df_triad_0)):
for i in range(len(df_triad_0.columns)):
    init_triad = df_triad_0.iloc[0, i]
    df_triad_0_normalized.iloc[:, i] = df_triad_0_normalized.iloc[:, i] - init_triad
    # for j in range(len(df_triad_0.columns)):
    #     df_triad_0_normalized.iloc[i, j] = df_triad_0_normalized.iloc[i, j] - init_triad

In [ ]:
plot_distance_wrtpath0(df_triad_0_normalized, geo, max_mapping_radius, Li_idxs = "all")

In [ ]:
geo = 1

df_triad_1_normalized = df_triad_1.copy()
# for i in range(len(df_triad_1)):
for i in range(len(df_triad_1.columns)):
    init_triad = df_triad_1.iloc[0, i]
    df_triad_1_normalized.iloc[:, i] = df_triad_1_normalized.iloc[:, i] - init_triad

In [ ]:
plot_distance_wrtpath0(df_triad_1_normalized, geo, max_mapping_radius, Li_idxs = "all")

# Check fullness of triads

In [ ]:
# geo = 1
# max_mapping_radius = 0.1
# destination_directory = f"restructure_{max_mapping_radius}/"

# # df_idx_triad_counts = pd.DataFrame#(np.zeros((24, 1)))
# # df_idx_triad_counts["idx_triad_counts"] = None

# if geo == 0:
#     path_geo = path_geo_0
# elif geo == 1:
#     path_geo = path_geo_1
# elif geo == 2:
#     path_geo = path_geo_2
# elif geo == 3:
#     path_geo = path_geo_3
# elif geo == 4:
#     path_geo = path_geo_4
# elif geo == 5:
#     path_geo = path_geo_5
# elif geo == 6:
#     path_geo = path_geo_6
# elif geo == 7:
#     path_geo = path_geo_7
# elif geo == 8:
#     path_geo = path_geo_8

# df_idx_triad_counts = pd.DataFrame(np.zeros((24, len(path_geo))))

# file_ref_24 = f"{geo}_0_{var_filename}.cif"
# file_path_ref_24 = os.path.join(destination_directory, file_ref_24)

# file_ref_72 = f"{filename_ref_72}.cif"
# file_path_ref_72 = os.path.join(destination_directory, file_ref_72)

# idx_coor_Li_dict_ref_24 = get_idx_coor_Li_dict(file_path_ref_24)    # key is the pointer to 24
# idx_coor_Li_dict_ref_72 = get_idx_coor_Li_dict(file_path_ref_72)    # key is the pointer to 24

# idx_coor_Li_dict_ref_triad = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_ref_24, idx_coor_Li_dict_ref_72)
# # idx_coor_Li_dict_ref_24 = get_idx_coor_Li_dict(file_path_ref_24)    # key is the pointer to 24
# # idxs_Li_ref_24 = list(idx_coor_Li_dict_ref_24.keys())
# # idx_coor_Li_dict_ref_72 = get_idx_coor_Li_dict(file_path_ref_72)    # key is the pointer to 24
# # idxs_Li_ref_72 = list(idx_coor_Li_dict_ref_72.keys())

# # idx_coor_Li_dict_ref_triad = defaultdict(list)

# # for key_72, coor_72 in idx_coor_Li_dict_ref_72.items():
# #     for key_24, coor_24 in idx_coor_Li_dict_ref_24.items():
# #         distance = mic_eucledian_distance(coor_72, coor_24)
# #         if distance == 0:
# #             idx_coor_Li_dict_ref_triad[key_24].append(coor_72)

# # for key_72, coor_72 in idx_coor_Li_dict_ref_72.items():
# #     for key_24, coor_24 in idx_coor_Li_dict_ref_24.items():
# #         distance = mic_eucledian_distance(coor_72, coor_24)
# #         if distance <= 0.086399 and distance != 0:                      # to edit this number
# #             idx_coor_Li_dict_ref_triad[key_24].append(coor_72)

# # for i in path_geo:
# for i in [3]:
#     # coor_Li = []
#     file = f"{geo}_{i}_{var_filename}.cif"
#     file_path = os.path.join(destination_directory, file)

#     idx_coor_Li_dict = get_idx_coor_Li_dict(file_path)

#     file_weirdos_appended = f"{geo}_{i}_{var_filename}_weirdos_appended.cif"
#     file_path_weirdos_appended = os.path.join(destination_directory, file_weirdos_appended)

#     idx_coor_Li_dict_weirdos_appended = get_idx_coor_Li_dict(file_path_weirdos_appended)
#     # idx_coor_Li_dict_ref_triad_weirdos_appended = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_weirdos_appended, idx_coor_Li_dict_ref_72)

#     idxs_Li = list(idx_coor_Li_dict.keys())
#     idxs_Li_not = sorted(i for i in range(24) if i not in idxs_Li)

#     # # idx_coor_Li_triad_belonging_initial = defaultdict(list)
#     # # idx_coor_Li_triad_belonging_initial_centroid = defaultdict(list)

#     ### does the numeration of Li is important?
#     ### 1) check which triad it does belong to initially

#     idx_coor_Li_idx_centroid_triad_ref = get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_ref_24)
#     idx_coor_Li_idx_centroid_triad = get_idx_coor_Li_idx_centroid_triad_w_closest_dist(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict)
#     idx_coor_Li_idx_centroid_triad_weirdos_appended = get_idx_coor_Li_idx_centroid_triad_w_closest_dist_weirdos_appended(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended, idxs_Li_not)
#     # idx_coor_Li_idx_centroid_triad_weirdos_appended = get_idx_coor_Li_idx_centroid_triad_w_closest_dist(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended)
#     idxs_Li_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad.keys()]
    
#     idx_triad_array = sorted([val['idx_triad'] for val in idx_coor_Li_idx_centroid_triad.values()])
#     idx_triad_array_not = sorted(i for i in range(24) if i not in idx_triad_array)
#     # idxs_Li_triad_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad()]

#     # idx_triad_series = pd.Series(idx_triad_array)
#     # df_idx_triad_counts[i] = idx_triad_series.value_counts()

#     idx_triad_counts = defaultdict(int)
#     # Count the occurrences of each idx_triad
#     # idx_triad_counts = pd.DataFrame(np.zeros((24, 1)))
#     for key, val in idx_coor_Li_idx_centroid_triad.items():
#         idx_triad = val['idx_triad']
#         idx_triad_counts[idx_triad] += 1
#     for j in idx_triad_array_not:
#         idx_triad_counts[j] = 0

#     # df_idx_triad_counts.at[i, "idx_triad_counts"] = dict(idx_triad_counts)
#     # df_idx_triad_counts = pd.DataFrame(np.zeros((24, 1)))
#     df_idx_triad_counts[i] = dict(idx_triad_counts)
#     # df_idx_triad_counts[i].fillna(0)

In [ ]:
# geo = 1
# max_mapping_radius = 0.1
# destination_directory = f"restructure_{max_mapping_radius}/"

# # df_idx_triad_counts = pd.DataFrame#(np.zeros((24, 1)))
# # df_idx_triad_counts["idx_triad_counts"] = None

# if geo == 0:
#     path_geo = path_geo_0
# elif geo == 1:
#     path_geo = path_geo_1
# elif geo == 2:
#     path_geo = path_geo_2
# elif geo == 3:
#     path_geo = path_geo_3
# elif geo == 4:
#     path_geo = path_geo_4
# elif geo == 5:
#     path_geo = path_geo_5
# elif geo == 6:
#     path_geo = path_geo_6
# elif geo == 7:
#     path_geo = path_geo_7
# elif geo == 8:
#     path_geo = path_geo_8

# df_idx_triad_counts = pd.DataFrame(np.zeros((24, len(path_geo))))

# file_ref_24 = f"{geo}_0_{var_filename}.cif"
# file_path_ref_24 = os.path.join(destination_directory, file_ref_24)

# file_ref_72 = f"{filename_ref_72}.cif"
# file_path_ref_72 = os.path.join(destination_directory, file_ref_72)

# idx_coor_Li_dict_ref_24 = get_idx_coor_Li_dict(file_path_ref_24)    # key is the pointer to 24
# idx_coor_Li_dict_ref_72 = get_idx_coor_Li_dict(file_path_ref_72)    # key is the pointer to 24

# idx_coor_Li_dict_ref_triad = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_ref_24, idx_coor_Li_dict_ref_72)
# # idx_coor_Li_dict_ref_24 = get_idx_coor_Li_dict(file_path_ref_24)    # key is the pointer to 24
# # idxs_Li_ref_24 = list(idx_coor_Li_dict_ref_24.keys())
# # idx_coor_Li_dict_ref_72 = get_idx_coor_Li_dict(file_path_ref_72)    # key is the pointer to 24
# # idxs_Li_ref_72 = list(idx_coor_Li_dict_ref_72.keys())

# # idx_coor_Li_dict_ref_triad = defaultdict(list)

# # for key_72, coor_72 in idx_coor_Li_dict_ref_72.items():
# #     for key_24, coor_24 in idx_coor_Li_dict_ref_24.items():
# #         distance = mic_eucledian_distance(coor_72, coor_24)
# #         if distance == 0:
# #             idx_coor_Li_dict_ref_triad[key_24].append(coor_72)

# # for key_72, coor_72 in idx_coor_Li_dict_ref_72.items():
# #     for key_24, coor_24 in idx_coor_Li_dict_ref_24.items():
# #         distance = mic_eucledian_distance(coor_72, coor_24)
# #         if distance <= 0.086399 and distance != 0:                      # to edit this number
# #             idx_coor_Li_dict_ref_triad[key_24].append(coor_72)

# # for i in path_geo:
# for i in [3]:
#     # coor_Li = []
#     file = f"{geo}_{i}_{var_filename}.cif"
#     file_path = os.path.join(destination_directory, file)

#     idx_coor_Li_dict = get_idx_coor_Li_dict(file_path)

#     file_weirdos_appended = f"{geo}_{i}_{var_filename}_weirdos_appended.cif"
#     file_path_weirdos_appended = os.path.join(destination_directory, file_weirdos_appended)

#     idx_coor_Li_dict_weirdos_appended = get_idx_coor_Li_dict(file_path_weirdos_appended)
#     # idx_coor_Li_dict_ref_triad_weirdos_appended = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_weirdos_appended, idx_coor_Li_dict_ref_72)

#     idxs_Li = list(idx_coor_Li_dict.keys())
#     idxs_Li_not = sorted(i for i in range(24) if i not in idxs_Li)

#     # # idx_coor_Li_triad_belonging_initial = defaultdict(list)
#     # # idx_coor_Li_triad_belonging_initial_centroid = defaultdict(list)

#     ### does the numeration of Li is important?
#     ### 1) check which triad it does belong to initially

#     idx_coor_Li_idx_centroid_triad_ref = get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_ref_24)
#     idx_coor_Li_idx_centroid_triad = get_idx_coor_Li_idx_centroid_triad_w_closest_dist(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict)
#     idx_coor_Li_idx_centroid_triad_weirdos_appended = get_idx_coor_Li_idx_centroid_triad_w_closest_dist_weirdos_appended(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended, idxs_Li_not)
#     # idx_coor_Li_idx_centroid_triad_weirdos_appended = get_idx_coor_Li_idx_centroid_triad_w_closest_dist(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended)
#     idxs_Li_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad.keys()]
    
#     idx_triad_array = sorted([val['idx_triad'] for val in idx_coor_Li_idx_centroid_triad.values()])
#     idx_triad_array_not = sorted(i for i in range(24) if i not in idx_triad_array)
#     # idxs_Li_triad_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad()]

#     # idx_triad_series = pd.Series(idx_triad_array)
#     # df_idx_triad_counts[i] = idx_triad_series.value_counts()

#     idx_triad_counts = defaultdict(int)
#     # Count the occurrences of each idx_triad
#     # idx_triad_counts = pd.DataFrame(np.zeros((24, 1)))
#     for key, val in idx_coor_Li_idx_centroid_triad.items():
#         idx_triad = val['idx_triad']
#         idx_triad_counts[idx_triad] += 1
#     for j in idx_triad_array_not:
#         idx_triad_counts[j] = 0

#     # df_idx_triad_counts.at[i, "idx_triad_counts"] = dict(idx_triad_counts)
#     # df_idx_triad_counts = pd.DataFrame(np.zeros((24, 1)))
#     df_idx_triad_counts[i] = dict(idx_triad_counts)
#     # df_idx_triad_counts[i].fillna(0)

In [ ]:
def get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72):
    # df_idx_triad_counts = pd.DataFrame #(np.zeros((24, 1)))
    # df_idx_triad_counts["idx_triad_counts"] = None

    idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = defaultdict(list)

    if geo == 0:
        path_geo = path_geo_0
    elif geo == 1:
        path_geo = path_geo_1
    elif geo == 2:
        path_geo = path_geo_2
    elif geo == 3:
        path_geo = path_geo_3
    elif geo == 4:
        path_geo = path_geo_4
    elif geo == 5:
        path_geo = path_geo_5
    elif geo == 6:
        path_geo = path_geo_6
    elif geo == 7:
        path_geo = path_geo_7
    elif geo == 8:
        path_geo = path_geo_8

    df_idx_triad_counts = pd.DataFrame(np.zeros((24, len(path_geo))))

    file_ref_24 = f"{geo}_0_{var_filename}.cif"
    file_path_ref_24 = os.path.join(destination_directory, file_ref_24)

    file_ref_72 = f"{filename_ref_72}.cif"
    file_path_ref_72 = os.path.join(destination_directory, file_ref_72)

    idx_coor_Li_dict_ref_24 = get_idx_coor_Li_dict(file_path_ref_24)    # key is the pointer to 24
    idx_coor_Li_dict_ref_72 = get_idx_coor_Li_dict(file_path_ref_72)    # key is the pointer to 24

    idx_coor_Li_dict_ref_triad = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_ref_24, idx_coor_Li_dict_ref_72)
    # idxs_Li_ref_24 = list(idx_coor_Li_dict_ref_24.keys())
    # idxs_Li_ref_72 = list(idx_coor_Li_dict_ref_72.keys())

    for i in path_geo:
        # coor_Li = []
        file = f"{geo}_{i}_{var_filename}.cif"
        file_path = os.path.join(destination_directory, file)

        idx_coor_Li_dict = get_idx_coor_Li_dict(file_path)

        file_weirdos_appended = f"{geo}_{i}_{var_filename}_weirdos_appended.cif"
        file_path_weirdos_appended = os.path.join(destination_directory, file_weirdos_appended)

        idx_coor_Li_dict_weirdos_appended = get_idx_coor_Li_dict(file_path_weirdos_appended)
        # idx_coor_Li_dict_ref_triad_weirdos_appended = get_idx_coor_Li_dict_ref_triad(idx_coor_Li_dict_weirdos_appended, idx_coor_Li_dict_ref_72)

        idxs_Li = list(idx_coor_Li_dict.keys())
        idxs_Li_not = sorted(i for i in range(24) if i not in idxs_Li)
        # idxs_Li = list(idx_coor_Li_dict.keys())

        # # idx_coor_Li_triad_belonging_initial = defaultdict(list)
        # # idx_coor_Li_triad_belonging_initial_centroid = defaultdict(list)

        ### does the numeration of Li is important?
        ### 1) check which triad it does belong to initially

        idx_coor_Li_idx_centroid_triad_ref = get_idx_coor_Li_idx_centroid_triad(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_ref_24)
        idx_coor_Li_idx_centroid_triad = get_idx_coor_Li_idx_centroid_triad_w_closest_dist(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict)
        idx_coor_Li_idx_centroid_triad_weirdos_appended = get_idx_coor_Li_idx_centroid_triad_w_closest_dist_weirdos_appended(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended, idxs_Li_not)
        # idx_coor_Li_idx_centroid_triad_weirdos_appended = get_idx_coor_Li_idx_centroid_triad_w_closest_dist(idx_coor_Li_dict_ref_triad, idx_coor_Li_dict_weirdos_appended)
        idxs_Li_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad.keys()]
        
        idx_triad_array = sorted([val['idx_triad'] for val in idx_coor_Li_idx_centroid_triad.values()])
        idx_triad_array_not = sorted(i for i in range(24) if i not in idx_triad_array)
        # idxs_Li_triad_dict = [i for i in range(24) if i in idx_coor_Li_idx_centroid_triad()]

        # idx_triad_series = pd.Series(idx_triad_array)
        # df_idx_triad_counts[i] = idx_triad_series.value_counts()

        idx_triad_counts = defaultdict(int)
        # Count the occurrences of each idx_triad
        # idx_triad_counts = pd.DataFrame(np.zeros((24, 1)))
        for key, val in idx_coor_Li_idx_centroid_triad.items():
            idx_triad = val['idx_triad']
            idx_triad_counts[idx_triad] += 1
        for j in idx_triad_array_not:
            idx_triad_counts[j] = 0

        # df_idx_triad_counts.at[i, "idx_triad_counts"] = dict(idx_triad_counts)
        # df_idx_triad_counts = pd.DataFrame(np.zeros((24, 1)))
        df_idx_triad_counts[i] = dict(idx_triad_counts)
        # df_idx_triad_counts[i].fillna(0)

        idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[i] = dict(idx_coor_Li_idx_centroid_triad_weirdos_appended)

    return df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict

In [ ]:
def get_idxs_val(df, val):
    idxs = [(row, col) for row in range(df.shape[0]) for col in range(df.shape[1]) if df.iloc[row, col] == val]
    return idxs


def get_key_Li_idx(dict, path, idx_triad):
    idxs_li = [key for key, value in dict[path].items() if value.get('idx_triad') == idx_triad]
    return idxs_li

In [ ]:
max_mapping_radius = 0.09
destination_directory = f"restructure_{max_mapping_radius}/"
var_filename = "mapLi_reindexed"
filename_ref_72 = "Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA_expanded"

In [ ]:
geo = 0
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

In [ ]:
idxs_0 = get_idxs_val(df_idx_triad_counts, 0)
idxs_2 = get_idxs_val(df_idx_triad_counts, 2)
print(f'idxs_0: {idxs_0}, idxs_2: {idxs_2}')

In [ ]:
geo = 1
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

In [ ]:
idxs_0 = get_idxs_val(df_idx_triad_counts, 0)
idxs_2 = get_idxs_val(df_idx_triad_counts, 2)
print(f'idxs_0: {idxs_0}, idxs_2: {idxs_2}')

In [ ]:
idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[3]

In [ ]:
idxs_li_triad20 = get_key_Li_idx(idx_coor_Li_idx_centroid_triad_weirdos_appended_dict, 3, 20)
print(f'idxs_li_triad20: {idxs_li_triad20}')

In [ ]:
geo = 2
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

weird, because supposedly 2/1, 2/3 has 1 weirdo but 2/1 shows no missing ion

In [ ]:
idxs_0 = get_idxs_val(df_idx_triad_counts, 0)
idxs_2 = get_idxs_val(df_idx_triad_counts, 2)
print(f'idxs_0: {idxs_0}, idxs_2: {idxs_2}')

In [ ]:
idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[3]

In [ ]:
idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[4]

In [ ]:
idxs_li_triad20 = get_key_Li_idx(idx_coor_Li_idx_centroid_triad_weirdos_appended_dict, 3, 20)
print(f'idxs_li_triad20: {idxs_li_triad20}')

In [ ]:
geo = 3
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

3/5 shows 3 missing atoms

In [ ]:
idxs_0 = get_idxs_val(df_idx_triad_counts, 0)
idxs_2 = get_idxs_val(df_idx_triad_counts, 2)
print(f'idxs_0: {idxs_0}, idxs_2: {idxs_2}')

In [ ]:
idxs_li_triad20 = get_key_Li_idx(idx_coor_Li_idx_centroid_triad_weirdos_appended_dict, 3, 20)
print(f'idxs_li_triad20: {idxs_li_triad20}')

In [ ]:
idxs_li_triad20 = get_key_Li_idx(idx_coor_Li_idx_centroid_triad_weirdos_appended_dict, 4, 13)
print(f'idxs_li_triad20: {idxs_li_triad20}')

In [ ]:
idxs_li_triad20 = get_key_Li_idx(idx_coor_Li_idx_centroid_triad_weirdos_appended_dict, 7, 20)
print(f'idxs_li_triad20: {idxs_li_triad20}')

In [ ]:
idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[3]

In [ ]:
idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[4]

In [ ]:
idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[5]

In [ ]:
idx_coor_Li_idx_centroid_triad_weirdos_appended_dict[7]

In [ ]:
geo = 4
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

In [ ]:
geo = 5
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

In [ ]:
geo = 6
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

In [ ]:
geo = 7
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts

In [ ]:
geo = 8
df_idx_triad_counts, idx_coor_Li_idx_centroid_triad_weirdos_appended_dict = get_triads_fullness(destination_directory, geo, var_filename, filename_ref_72)

In [ ]:
df_idx_triad_counts